# Parabolic model problem
We are solving the unsteady heat equation 

$$\text{find: } u(t) \in H_{0,D}^1 \quad m(\partial_t u, v) + a(u,v) = f(v) \quad \forall v \in H_{0,D}^1, \quad t \in (0,T), \quad u(t=0) = u_0$$
with the multilinear forms
$$ m(u,v) = \int_{\Omega} u v, \quad a(u,v) = \int_{\Omega} \nabla u \nabla v + b \cdot \nabla u v \quad \text{ and } \quad f(v) = \int_{\Omega} f v $$
where $b$ is the wind $b = (2y(1-x^2),-2x(1-y^2))$.

import NGSolve and Netgen Python modules:

In [ ]:
import netgen.gui
%gui tk
from ngsolve import *
from netgen.geom2d import SplineGeometry

Generate an unstructured mesh of maximal mesh-size 0.2:

In [ ]:
geo = SplineGeometry()
geo.AddRectangle( (-1, -1), (1, 1), bcs = ("bottom", "right", "top", "left"))
mesh = Mesh( geo.GenerateMesh(maxh=0.25))

Declare a finite element space:

In [ ]:
fes = H1(mesh, order=3, dirichlet="bottom|right|left|top")

Declare test and trial functions

In [ ]:
u = fes.TrialFunction()
v = fes.TestFunction()

define and assemble linear and bilinear forms. We do this seperately for the mass matrix and the stiffness matrix:

In [ ]:
b = CoefficientFunction((2*y*(1-x*x),-2*x*(1-y*y)))
Draw(b,mesh,"wind")

In [ ]:
a = BilinearForm(fes, symmetric=False)
a += SymbolicBFI (0.01*grad(u)*grad(v) + b*grad(u)*v)
a.Assemble()

m = BilinearForm(fes, symmetric=False)
m += SymbolicBFI (u*v)
m.Assemble()

We want to use a simple implicit Euler time stepping method, i.e.

$$
  M + \Delta t A u^{n+1} = M u^n + f^{n+1}
$$

or in an incremental form:

$$
  M + \Delta t A (u^{n+1} - u^n) = - \Delta t A u^n + f^{n+1}.
$$

The incremental form has the advantage that $u^{n+1} - u^n$ has homogeneous boundary conditions (unless boundary conditions are time-dependent).

In [ ]:
dt = 0.001

For the time stepping method we want to set up linear combinations of matrices.

(Only) if the sparsity pattern of the matrices agree we can copy the pattern and sum up the entries.

First, we create a matrix of the same size and sparsity pattern as m.mat:

In [ ]:
mstar = m.mat.CreateMatrix()

In [ ]:
print(mstar)

To access the entries we use the vector of nonzero-entries:

In [ ]:
print(mstar.AsVector())

In [ ]:
mstar.AsVector().data = m.mat.AsVector() + dt * a.mat.AsVector()

In [ ]:
invmat = mstar.Inverse(freedofs=fes.FreeDofs())

We set the initial data: $u_0 = (1-y^2)x$

In [ ]:
gfu = GridFunction(fes)
gfu.Set((1-y*y)*x)
Draw(gfu,mesh,"u")

And r.h.s. $f = exp(-6 ((x+\frac12)^2+y^2)) - exp(-6 ((x-\frac12)^2+y^2))$

In [ ]:
f = LinearForm(fes)
gaussp = exp(-6*((x+0.5)*(x+0.5)+y*y))-exp(-6*((x-0.5)*(x-0.5)+y*y))
Draw(gaussp,mesh,"f")
f += SymbolicLFI(gaussp*v)
f.Assemble()

In [ ]:
rhs = gfu.vec.CreateVector()
rhs[:] = 0

In [ ]:
Draw(gfu,mesh,"u")

In [ ]:
t = 0.0

In [ ]:
tstep = 0.2

In [ ]:
t1=0
while t1 < tstep - 0.5 * dt:
    rhs.data = dt * f.vec - dt * a.mat * gfu.vec
    gfu.vec.data += invmat * rhs
    t1 += dt
    print("\r",t+t1,end="")
    Redraw(blocking=False)
print("")
t+=t1